In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

Note need rotation matrices and coords for this to work

In [ ]:
arrs = [np.array([1,2,3]),np.array([1,2,3,4]), np.array([1,2,3,4,5]),np.array([1,2,3]),np.array([1,2,3,4]), np.array([1,2,3,4,5])]
# arrs = split_points1D.copy()
print(arrs[1])

n_edges = len(arrs)
max_l = max(len(arr) for arr in arrs)

test_arr = np.full((n_edges,max_l,2),np.nan)
t=time.perf_counter()

for i, arr in enumerate(arrs):
    zeros = np.zeros(arr.shape[0])
    arr2d = np.column_stack((arr,zeros))
    #arr.shape[0] is the number of points
    test_arr[i,:arr.shape[0]] = arr2d


# print(test_arr[1])
rotated = rotation_matrices@test_arr.transpose((0,2,1))
rot = rotated.transpose((0,2,1))
translated = coords[:-1][:, np.newaxis, :]+rot
t=time.perf_counter()-t
print(f"{t=}")
# print(rot[1])
# print(coords[:-1][:, np.newaxis, :])
# print(translated)
# print(rotation_matrices[2])
# print(rot[2])


In [ ]:
# Flatten the array and filter out 'nan' values
print(translated.shape)
x = translated[:,:,0].flatten()
y = translated[:,:,1].flatten()
valid_indices = ~np.isnan(x) & ~np.isnan(y)
x = x[valid_indices]
print(x.shape)
y = y[valid_indices]

# Create scatter plot
plt.scatter(x, y)
plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')
plt.title('Scatter Plot of Data Points')
plt.show()

In [ ]:
import numpy as np

class CascadingArraysTest:
    def __init__(self, array, max_values):
        self.array = array
        self.max_values = max_values

    def add(self, value):
        # Increment all elements
        self.array += value

        # Then cascade
        for i in range(len(self.max_values)):
            self.cascade(i)

    def cascade(self, index):
        current_index = index
        while True:
            sub_array = self.array[current_index]
            overflow = sub_array > self.max_values[current_index]
            if not np.any(overflow):
                break

            overflow_values = sub_array[overflow] - self.max_values[current_index]
            sub_array[overflow] = np.nan  # Replace overflow values with nan in the current sub-array

            # Cascade overflow values to the next sub-array
            next_index = (current_index + 1) % len(self.max_values)
            next_sub_array = self.array[next_index]
            free_indices = np.where(np.isnan(next_sub_array))[0]

            # Number of values we can move to the next array
            num_movable_values = min(len(free_indices), len(overflow_values))

            # Move values to the next array
            next_sub_array[free_indices[:num_movable_values]] = overflow_values[:num_movable_values]

            # Any remaining overflow values need further cascading
            overflow_values = overflow_values[num_movable_values:]

            # Update the original arrays
            self.array[current_index] = sub_array
            self.array[next_index] = next_sub_array

            if overflow_values.size > 0:
                # Prepare for next iteration
                self.array[next_index][-len(overflow_values):] = overflow_values
                current_index = next_index
            else:
                break

# Example usage
array = np.array([
    np.pad(np.array([1.0, 2.0]), (0, 2), mode='constant', constant_values=np.nan),
    np.pad(np.array([3.0, 4.0]), (0, 2), mode='constant', constant_values=np.nan),
    np.array([5.0, 6.0, np.nan, np.nan])
])
max_values = [3, 5, 7]
cascading_arrays = CascadingArraysTest(array, max_values)

# Add a value
t= time.perf_counter()
cascading_arrays.add(2)
print(time.perf_counter()-t)
# Check the array after addition
print(cascading_arrays.array)
